<a href="https://colab.research.google.com/github/aditinambur/SmartDoc-Q-A/blob/main/SmartDoc_Q_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers transformers accelerate PyMuPDF faiss-cpu torch torchvision torchaudio rank_bm25
!pip install numpy<2 --upgrade
!pip install -q gradio

/bin/bash: line 1: 2: No such file or directory


In [2]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from google.colab import files
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz
import numpy as np
import faiss
from transformers import pipeline
from google.colab import userdata
import os
import torch
import gradio as gr
from rank_bm25 import BM25Okapi
import nltk
import os
import tempfile


In [3]:
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
from google.colab import userdata
import google.generativeai as genai

gemini_api_key = userdata.get('GEMINI_API_KEY')

genai.configure(api_key=gemini_api_key)

gemini_model = genai.GenerativeModel('gemini-1.5-flash')

print("Gemini API configured successfully!")

Gemini API configured successfully!


In [6]:
chat_chunks = []
chat_embeddings = []
chat_index = faiss.IndexFlatL2(384) #dimension 384
chat_bm25_corpus = []
chat_bm25_index = None

def passability_check(query, threshold=0.7):
    if not chat_chunks:
        return False

    query_vec = model.encode([query]).astype('float32')
    D, _ = chat_index.search(query_vec, k=1)
    best_sim = 1 - (D[0][0] / 2)  #convert L2 to cosine-like approximation

    return best_sim >= threshold


def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def embed(text):
    x = max(300, min(1000, int(len(text) / 8)))
    y = max(50, int(0.15 * x))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=x, chunk_overlap=y)
    chunks = text_splitter.split_text(text)
    print(f"Created {len(chunks)} chunks")
    chunk_embeddings = model.encode(chunks)
    embedding_matrix = np.array(chunk_embeddings).astype('float32')
    print(f"Embedding matrix shape: {embedding_matrix.shape}")
    index = faiss.IndexFlatL2(embedding_matrix.shape[1])
    index.add(embedding_matrix)
    tokenized_corpus = [nltk.word_tokenize(doc) for doc in chunks]
    bm25 = BM25Okapi(tokenized_corpus)
    return chunks, index, bm25

def gen_prompt(chunks, query):
    context = "\n".join(chunks)
    prompt = f"""You are a helpful assistant answering based *only* on the following context (from documents or past conversation).

Context: {context}

Question: {query}

Answer:"""
    return prompt

def build_prompt(question):
    return f"""Classify the following question into one of these categories specified with numbers:
1-– asks for a specific fact (e.g., "When was the Constitution signed?") - 1
2-– can be answered with yes or no (e.g., "Can a contract be voided under duress?") - 2
6-– asks for a detailed explanation (e.g., "How does a merger affect minority shareholders?") - 6
8-- summary – asks to summarize a document, event, or content (e.g., "Summarize the key terms of this agreement.") - 8


Here are a few examples:

Q: Who is the current president of India?
A: 1

Q: Is the law still valid in 2025?
A: 2

Q: What is inflation?
A: 6

Q: Summarize the main points of the introduction section.
A: 8

Now classify the next question and return either 1/2/6/8:

Q: {question}
A:"""

def classify_question(question):
    prompt = build_prompt(question)

    try:
        response = gemini_model.generate_content(prompt)
        answer = response.text.strip().split()[0]
        answer_num = int(''.join(filter(str.isdigit, answer)))

        if answer_num in [1, 2, 6, 8]:
            return answer_num
        else:
            return 4
    except Exception as e:
        print(f"Classification error: {e}. Using default value.")
        return 4

def rag_answer(query, chunks, model, index, bm25_index, method, gen_model):
    global chat_bm25_index

    num_chunks = classify_question(query)
    max_chunks = len(chunks)
    num_chunks = min(num_chunks, max_chunks)

    if method == "RAG (Vector Search)":
        query_encoding = model.encode([query]).astype('float32')
        _, indices = index.search(query_encoding, k=num_chunks)
        chunk_contexts = [chunks[i] for i in indices[0] if i < len(chunks)]
    elif method == "BM25":
        tokenized_query = nltk.word_tokenize(query)
        scores = bm25_index.get_scores(tokenized_query)
        top_n = np.argsort(scores)[::-1][:num_chunks]
        chunk_contexts = [chunks[i] for i in top_n if i < len(chunks)]
    elif method == "Both":
        # RAG retrieval
        query_encoding = model.encode([query]).astype('float32')
        _, rag_indices = index.search(query_encoding, k=num_chunks)
        rag_contexts = [chunks[i] for i in rag_indices[0] if i < len(chunks)]

        # BM25 retrieval
        tokenized_query = nltk.word_tokenize(query)
        scores = bm25_index.get_scores(tokenized_query)
        bm25_top_n = np.argsort(scores)[::-1][:num_chunks]
        bm25_contexts = [chunks[i] for i in bm25_top_n if i < len(chunks)]

        # Combine and deduplicate contexts
        chunk_contexts = rag_contexts
        for context in bm25_contexts:
            if context not in chunk_contexts:
                chunk_contexts.append(context)
    else:
        return "Invalid retrieval method selected."

    if not chunk_contexts:
        return "No relevant context found for the query."

    history_chunks = []

    if passability_check(query):
        if method == "RAG (Vector Search)":
            query_encoding = model.encode([query]).astype('float32')
            _, indices = chat_index.search(query_encoding, k=2)
            history_chunks = [chat_chunks[i] for i in indices[0]]
        elif method == "BM25":
            if chat_bm25_index:
                tokenized_query = nltk.word_tokenize(query)
                scores = chat_bm25_index.get_scores(tokenized_query)
                top_n = np.argsort(scores)[::-1][:2]
                history_chunks = [chat_chunks[i] for i in top_n]
        elif method == "Both":
            query_encoding = model.encode([query]).astype('float32')
            _, rag_indices = chat_index.search(query_encoding, k=2)
            rag_chunks = [chat_chunks[i] for i in rag_indices[0]]

            if chat_bm25_index:
                tokenized_query = nltk.word_tokenize(query)
                scores = chat_bm25_index.get_scores(tokenized_query)
                bm25_top_n = np.argsort(scores)[::-1][:2]
                bm25_chunks = [chat_chunks[i] for i in bm25_top_n]
            else:
                bm25_chunks = []


            history_chunks = rag_chunks
            for chunk in bm25_chunks:
                if chunk not in history_chunks:
                    history_chunks.append(chunk)

    full_context = history_chunks + chunk_contexts
    prompt = gen_prompt(full_context, query)


    try:
        response = gen_model.generate_content(prompt)
        qa_pair = f"Q: {query}\nA: {response.text.strip()}"
        chat_chunks.append(qa_pair)

        if len(chat_chunks) > 20:
            chat_chunks.pop(0)
            chat_embeddings.pop(0)
            chat_bm25_corpus.pop(0)

        new_embedding = model.encode([qa_pair])[0].astype('float32')
        chat_embeddings.append(new_embedding)
        chat_index.reset()
        chat_index.add(np.array(chat_embeddings))

        chat_bm25_corpus.append(nltk.word_tokenize(qa_pair))
        chat_bm25_index = BM25Okapi(chat_bm25_corpus)


        return response.text.strip()
    except Exception as e:
        return f"Error generating response: {str(e)}"

def file_upload(file_obj):
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file_obj)
        tmp_file_path = tmp_file.name

    text = read_pdf(tmp_file_path)

    os.remove(tmp_file_path)

    chunks, index, bm25_index = embed(text)
    return chunks, index, bm25_index

def answer_with_rag(query, chunks, index, bm25_index, method):
    return rag_answer(query, chunks, model, index, bm25_index, method, gemini_model)

print("Core functions loaded successfully!")

Core functions loaded successfully!


In [8]:
with gr.Blocks(title="RAG Document Query with Gemini") as demo:
    gr.Markdown("## 📄 RAG Document Query with Gemini API")
    gr.Markdown("Upload a PDF document and ask questions about its content using different retrieval methods.")

    # State variables to hold chunks and index
    chunks_state = gr.State(None)
    index_state = gr.State(None)
    bm25_index_state = gr.State(None)

    # --- File Upload ---
    with gr.Row():
        file_input = gr.File(label="Upload your PDF document", file_types=[".pdf"])

    process_button = gr.Button("📥 Process Document", variant="primary")
    process_status = gr.Textbox(label="Processing Status", interactive=False)

    # --- Retrieval Method Selection ---
    retrieval_method = gr.Radio(
        choices=["RAG (Vector Search)", "BM25", "Both"],
        label="Select Retrieval Method",
        value="RAG (Vector Search)"
    )

    # --- Question + Answer ---
    with gr.Row():
        with gr.Column():
            query_input = gr.Textbox(
                lines=3,
                placeholder="Ask your question here...",
                label="📝 Question"
            )
            answer_button = gr.Button("🔍 Get Answer", variant="secondary")

        with gr.Column():
            output_text = gr.Textbox(
                label="📤 Answer",
                lines=8,
                interactive=False
            )

    # --- Actions ---
    def process_document(file_obj):
        if file_obj is None:
            return "Please upload a PDF file first.", None, None, None

        try:
            # Get the file path from the Gradio file object
            file_path = file_obj.name
            # Read the file content as bytes from the path
            with open(file_path, 'rb') as f:
                file_content = f.read()

            chunks, index, bm25_index = file_upload(file_content)
            return f"Document processed successfully! Created {len(chunks)} chunks.", chunks, index, bm25_index
        except Exception as e:
            return f"Error processing document: {str(e)}", None, None, None

    def get_answer(query, chunks, index, bm25_index, method):
        if chunks is None or index is None or bm25_index is None:
            return "Please process a document first."

        if not query.strip():
            return "Please enter a question."

        try:
            answer = answer_with_rag(query, chunks, index, bm25_index, method)
            return answer
        except Exception as e:
            return f"Error generating answer: {str(e)}"

    process_button.click(
        fn=process_document,
        inputs=[file_input],
        outputs=[process_status, chunks_state, index_state, bm25_index_state]
    )

    answer_button.click(
        fn=get_answer,
        inputs=[query_input, chunks_state, index_state, bm25_index_state, retrieval_method],
        outputs=output_text
    )

    # Some examples
    gr.Markdown("### Example Questions:")
    gr.Markdown("- What are the main points discussed in this document?")
    gr.Markdown("- Summarize the key findings.")
    gr.Markdown("- What is the conclusion of this document?")


demo.launch(share=True, debug=True)

print("Gradio interface launched!")

TypeError: Textbox.__init__() got an unexpected keyword argument 'allow_copy'